In [40]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from scipy.stats import burr, lognorm, expon
import matplotlib.pyplot as plt

In [41]:
class ElevatorSimulation:
  def __init__(self, policy, seed):
    # 1. Parameters
    self.elev_capacity = 10
    self.moving_time = 5.76
    self.passing_time = 2.67
    self.Q_LIMIT = 20
    self.time_end = 15*60 # 15min
    self.elev_policy = policy # All, Odd, Even, High, Low

    # Random number generation
    self.custs_arrival_gen = expon.rvs(loc=0.014535693, scale=6.863142713788785, size=1000, random_state=seed).tolist()
    self.custs_arrival_gen = [i for i in self.custs_arrival_gen if i >= 0 and i <60]
    self.elev_arrival_gen = lognorm.rvs(s=0.5982911680689725, loc=-1.474276076031476,
                                        scale=10.248234690476108, size=1000, random_state=seed).tolist()
    self.elev_arrival_gen = [i for i in self.elev_arrival_gen if i > 0 and i <70]
    self.stopping_time_gen = burr.rvs(c=2.5874763167784525, d=0.8045643981906201,
                                      loc=-0.6344383268940723, scale=21.480276631262164, size=1000,random_state=seed).tolist()
    self.stopping_time_gen = [i for i in self.stopping_time_gen if i >= 5 and i <90]

    self.floor = [2,3,4,5,6]
    self.floor_p = [0.02631578947,0.05789473684, 0.6210526316, 0.2315789474, 0.06315789474]

    # 2. Initialization
    self.initialize()

  def initialize(self):
    # Simulation clock
    self.sim_time = 0.0

    # State variables
    self.elevator_status = [0, 0, 0, 0, 0, 0]
    self.num_in_q = 0
    self.time_last_event = 0.0
    self.button_pressed = [False, False, False, False, False, False]
    self.custs_in_elev = [0, 0, 0, 0, 0, 0]

    # Statistical counters
    self.num_custs_delayed = 0
    self.total_in_waiting = 0.0
    self.total_in_service = 0.0
    self.area_num_in_waiting = 0.0
    self.area_elev_status = [0.0] * 6

    # Event list (customer_arrival, elevator_arrival, customer_departure, end_simulation)
    self.event_df = pd.DataFrame(columns=['event_time', 'event_type', 'demand_floor', 'elevator_number'])
    self.event_df.loc[0] = [self.sim_time + self.custs_arrival_gen.pop(0), 'customer_arrival', int(np.random.choice(self.floor,1,p=self.floor_p)), None]
    self.event_df.loc[1] = [1.0e+30, 'elevator_arrival', None, None]
    self.event_df.loc[2] = [1.0e+30, 'customer_departure', None, None]
    self.event_df.loc[3] = [self.time_end, 'end_simulation', None, None]

    # Queue
    self.waiting_q_df = pd.DataFrame(columns=['time_arrival', 'destination'])

  def timing(self):
    min_time_next_event = 1.0e+29
    next_event_type = ''

    # sort event_df
    self.event_df = self.event_df.sort_values(by='event_time', ascending=True)
    self.event_df.reset_index(drop=True, inplace=True)

    # Determine the event type of the next event to occur
    if self.event_df.iloc[0,0] < min_time_next_event:
      min_time_next_event = self.event_df.iloc[0, 0]
      next_event_type = self.event_df.iloc[0, 1]

    # check to see whether the event list is empty
    if next_event_type == '':
      print(f'Event list empty at time {self.sim_time}')
    else:
      pass

    self.next_event_type = next_event_type
    self.sim_time = min_time_next_event

  def customer_arrival(self):
    current_custs_demand = self.event_df.iloc[0, 2]

    # Schedule next arrival & determine the customer's destination
    self.event_df.loc[len(self.event_df)] = [self.sim_time + self.custs_arrival_gen.pop(0), 'customer_arrival', int(np.random.choice(self.floor,1,p=self.floor_p)), None]

    # Increase waiting queue
    if self.num_in_q < self.Q_LIMIT:
      self.num_in_q += 1
    else:
      print(f'Overflow of the customer_arrival at time {self.sim_time}')
      return


    self.waiting_q_df.loc[len(self.waiting_q_df)] = [self.sim_time, current_custs_demand]
    self.waiting_q_df = self.waiting_q_df.sort_values('time_arrival', ascending=True)
    self.waiting_q_df.reset_index(drop=True, inplace=True)
    # Invoke button_pressor_evaluation
    self.button_pressor_evaluation()

  def find_available_n_button_pressed(self, destination):
    All = [2, 3, 4, 5, 6]
    Low = [2, 3, 4]
    High = [5, 6]
    Odd = [3, 5]
    Even = [2, 4, 6]

    if destination in Low:
      available_idx = [idx for idx, value in enumerate(self.elev_policy) if value in ['All', 'Low']]
      button_pressed_idx = [idx for idx, value in enumerate(self.button_pressed) if value]
    elif destination in High:
      available_idx = [idx for idx, value in enumerate(self.elev_policy) if value in ['All', 'High']]
      button_pressed_idx = [idx for idx, value in enumerate(self.button_pressed) if value]
    if destination in Odd:
      available_idx += [idx for idx, value in enumerate(self.elev_policy) if value == 'Odd']
    elif destination in Even:
      available_idx += [idx for idx, value in enumerate(self.elev_policy) if value == 'Even']

    available_n_button_pressed = list(set(available_idx) & set(button_pressed_idx))
    return available_idx, button_pressed_idx, available_n_button_pressed

  def button_pressor_evaluation(self):
    for idx, i in enumerate(self.waiting_q_df.iloc[:,1]):
      available_idx, button_pressed_idx, available_n_button_pressed = self.find_available_n_button_pressed(i)
      available_n_button_not_pressed = list(set(available_idx) - set(button_pressed_idx))
      # Checkpoint1
      # No
      if len(available_n_button_pressed) == 0:
        # Press button
        choosed_elev = random.choice(available_n_button_not_pressed)
        self.button_pressed[choosed_elev] = True
        # Schedule elevator arrival event
        self.event_df.loc[len(self.event_df)] = [self.sim_time + self.elev_arrival_gen.pop(0), 'elevator_arrival', None, choosed_elev]
        self.event_df = self.event_df.sort_values('event_time', ascending=True)
        self.event_df.reset_index(drop=True, inplace = True)
      # Yes
      else:
        # check how many customers before the customer
        cnt = 0
        for index, customer in self.waiting_q_df.iterrows():
          if idx == index:
            break
          cnt +=1
        # Checkpoint2
        # No
        if cnt >= 5:
          # Checkpoint3
          # Yes
          if len(available_n_button_not_pressed) > 0:
            # Press button
            choosed_elev = random.choice(available_n_button_not_pressed)
            self.button_pressed[choosed_elev] = True
            # Schedule elevator arrival event
            self.event_df.loc[len(self.event_df)] = [self.sim_time + self.elev_arrival_gen.pop(0), 'elevator_arrival', None, choosed_elev]
            self.event_df = self.event_df.sort_values('event_time', ascending=True)
            self.event_df.reset_index(drop=True, inplace = True)


  def elevator_arrival(self):
    elev_no = self.event_df.iloc[0, 3]
    customer_in_elev = 0
    stop_floor = []

    # Make the elevator busy
    self.elevator_status[elev_no] = 1
    # Make the button unpressed
    self.button_pressed[elev_no] = False

    # Select customers who will take the elevator
    available_floor = self.get_available_floors(elev_no)
    for index, customer in self.waiting_q_df.iterrows():
      if customer_in_elev == self.elev_capacity:
        break
      destination = customer['destination']
      if destination in available_floor:
        self.num_in_q -= 1
        customer_in_elev += 1
        stop_floor.append(destination)
        time_custs_arrival = customer['time_arrival']
        waiting = self.sim_time - time_custs_arrival
        self.total_in_waiting += waiting

        # Drop customers in the queue
        self.waiting_q_df = self.waiting_q_df.iloc[1:,:]
        self.waiting_q_df.reset_index(drop=True, inplace=True)

        # Calculate service time
        service_time = []
        stop_floor_unique = sorted(list(set(stop_floor)))
        service_time_unique = {}
        last_floor = 1
        for floor in stop_floor_unique:
          if last_floor == 1:
            passing_num = floor - 2
          else:
            passing_num = floor - last_floor
          service_time_unique[floor] = self.stopping_time_gen.pop(0) + self.passing_time*passing_num
          last_floor = floor

        for floor in stop_floor:
            service_time.append(service_time_unique[floor])
        maximum_service_time = max(service_time)

        service_time.append(self.moving_time)
        self.total_in_service += sum(service_time)

        # Schedule customer departure event
        self.event_df.loc[len(self.event_df)] = [self.sim_time + maximum_service_time, 'customer_departure', None, elev_no]
        self.event_df = self.event_df.sort_values('event_time', ascending=True)
        self.event_df.reset_index(drop=True, inplace = True)

    # Invoke button pressor evaluation
    self.button_pressor_evaluation()

    # Update custs_in_elev
    self.custs_in_elev[elev_no] = customer_in_elev

  def get_available_floors(self, elev_no):
    available_floor = []
    elev_policy = self.elev_policy[elev_no]
    if elev_policy == 'All':
      available_floor = [2, 3, 4, 5, 6]
    elif elev_policy == 'High':
      available_floor = [5, 6]
    elif elev_policy == 'Low':
      available_floor = [2,3,4]
    elif elev_policy == 'Odd':
      available_floor = [3, 5]
    elif elev_policy == 'Even':
      available_floor = [2, 4, 6]
    return available_floor

  def customer_departure(self):
    elev_no = self.event_df.iloc[0, 3]
    # record num_custs_delayed
    self.num_custs_delayed += self.custs_in_elev[elev_no]

    # Make the elevator idle
    self.elevator_status[elev_no] = 0

    # Make custs_in_elev to zero
    self.custs_in_elev[elev_no] = 0


  def update_time_avg_stats(self):
    time_since_last_event = self.sim_time - self.time_last_event
    self.time_last_event = self.sim_time

    # Update area under the number-in-waiting-line function
    self.area_num_in_waiting += self.num_in_q * time_since_last_event

    # Update area under elevator-busy indicator function
    for i in range(6):
      self.area_elev_status[i] += self.elevator_status[i] * time_since_last_event

  def report(self):
    '''
    print('---------------------------------------------------------------------')
    print(f'Average waiting time in GF: {self.total_in_waiting / self.num_custs_delayed}')
    print(f'Average service time in elevator: {self.total_in_service / self.num_custs_delayed}')
    print(f'Average total time in elevator: {(self.total_in_waiting + self.total_in_service) / self.num_custs_delayed}')

    print(f'Average number in waiting queue: {self.area_num_in_waiting / self.sim_time}')

    elev_utilization = [self.area_elev_status[i] / self.sim_time for i in range(6)]
    for i in range(6):
      print(f'Elevator {i + 1} utilization: {elev_utilization[i]}')

    avg_elev_utilization = sum(elev_utilization) / 6
    print(f'Average elevator utilization: {avg_elev_utilization}')
    print('---------------------------------------------------------------------')
    '''

  def save_results(self):
    df = pd.DataFrame(columns=['avg_waiting_time', 'avg_service_time', 'avg_total_time', 'avg_num_in_q',
                               'elev1_utilization', 'elev2_utilization', 'elev3_utilization', 'elev4_utilization', 'elev5_utilization', 'elev6_utilization',
                               'avg_elev_utilization'])
    avg_waiting_time = self.total_in_waiting / self.num_custs_delayed
    avg_service_time = self.total_in_service / self.num_custs_delayed
    avg_total_time = (self.total_in_waiting + self.total_in_service) / self.num_custs_delayed
    avg_num_in_q = self.area_num_in_waiting / self.sim_time
    elev_utilization = [self.area_elev_status[i] / self.sim_time for i in range(6)]
    avg_elev_utilization = sum(elev_utilization) / 6
    df.loc[0] = [avg_waiting_time, avg_service_time, avg_total_time, avg_num_in_q,
                 elev_utilization[0], elev_utilization[1], elev_utilization[2], elev_utilization[3], elev_utilization[4], elev_utilization[5],
                 avg_elev_utilization]
    return df


  def run_simulation(self):
    # 3. Run the simulation while more delays are still needed
    while 1:
      # Determine next event
      self.timing()
      # Update time-average statistical accumulators
      self.update_time_avg_stats()
      # Invoke the appropriate event function
      if self.next_event_type == 'customer_arrival':
        self.customer_arrival()
        self.event_df = self.event_df.iloc[1:,:] # delete first row
        self.event_df = self.event_df.sort_values('event_time', ascending=True)
        self.event_df.reset_index(inplace = True, drop=True)
      elif self.next_event_type == 'elevator_arrival':
        self.elevator_arrival()
        self.event_df = self.event_df.iloc[1:,:] # delete first row
        self.event_df = self.event_df.sort_values('event_time', ascending=True)
        self.event_df.reset_index(inplace = True, drop=True)
      elif self.next_event_type == 'customer_departure':
        self.customer_departure()
        self.event_df = self.event_df.iloc[1:,:] # delete first row
        self.event_df = self.event_df.sort_values('event_time', ascending=True)
        self.event_df.reset_index(inplace = True, drop=True)
      elif self.next_event_type == 'end_simulation':
        # 4. Invoke the report generator and end the simulation
        self.report()
        self.result = self.save_results()
        break




In [42]:
class experiment:
  def __init__(self, policy_list, repeat_num):
    self.policy_list = policy_list
    self.repeat_num = repeat_num
    self.seeds = np.random.choice(range(1, 10000), repeat_num, replace=False)
    self.results = None

  def run(self):
    for idx, policy in enumerate(tqdm(self.policy_list)):
      print('\n')
      for j in range(self.repeat_num):
        if j%10 == 0:
          print(f'{j+1} / {self.repeat_num} is processing')
        self.policy = policy
        self.seed = self.seeds[j]
        elevator_sim = ElevatorSimulation(policy=self.policy, seed = self.seed)
        elevator_sim.run_simulation()
        self.result_tmp = elevator_sim.result
        self.result_tmp['policy'] = idx
        self.result_tmp['seed'] = self.seed

        if self.results is None:
          self.results = self.result_tmp
        else:
          self.results = pd.concat([self.results, self.result_tmp], axis=0)





*   6개 다 전층
*   6개 다 짝홀
*   6개 다 고저
*   2개 전층 4개 짝홀
*   2개 전층 4개 고저
*   4개 전층 2개 짝홀
*   4개 전층 2개 고저
*   2개 전층 2개 짝홀 2개 고저


In [43]:
policy_list = [['All', 'All', 'All', 'All', 'All', 'All'],
               ['Odd', 'Even', 'Odd', 'Even', 'Odd', 'Even'],
               ['High', 'Low', 'High', 'Low', 'High', 'Low'],
               ['All', 'All', 'Odd', 'Even', 'Odd', 'Even'],
               ['All', 'All', 'High', 'Low', 'High', 'Low'],
               ['All', 'All', 'All', 'All', 'Odd', 'Even'],
               ['All', 'All', 'All', 'All', 'High', 'Low'],
               ['All', 'All', 'High', 'Low', 'Odd', 'Even']]
repeat_num = 300

exp = experiment(policy_list, repeat_num)
exp.run()

  0%|          | 0/8 [00:00<?, ?it/s]



1 / 300 is processing
11 / 300 is processing
21 / 300 is processing
31 / 300 is processing
41 / 300 is processing
51 / 300 is processing
61 / 300 is processing
71 / 300 is processing
81 / 300 is processing
91 / 300 is processing
101 / 300 is processing
111 / 300 is processing
121 / 300 is processing
131 / 300 is processing
141 / 300 is processing
151 / 300 is processing
161 / 300 is processing
171 / 300 is processing
181 / 300 is processing
191 / 300 is processing
201 / 300 is processing
211 / 300 is processing
221 / 300 is processing
231 / 300 is processing
241 / 300 is processing
251 / 300 is processing
261 / 300 is processing
271 / 300 is processing
281 / 300 is processing
291 / 300 is processing


 12%|█▎        | 1/8 [05:50<40:55, 350.78s/it]



1 / 300 is processing
11 / 300 is processing
21 / 300 is processing
31 / 300 is processing
41 / 300 is processing
51 / 300 is processing
61 / 300 is processing
71 / 300 is processing
81 / 300 is processing
91 / 300 is processing
101 / 300 is processing
111 / 300 is processing
121 / 300 is processing
131 / 300 is processing
141 / 300 is processing
151 / 300 is processing
161 / 300 is processing
171 / 300 is processing
181 / 300 is processing
191 / 300 is processing
201 / 300 is processing
211 / 300 is processing
221 / 300 is processing
231 / 300 is processing
241 / 300 is processing
251 / 300 is processing
261 / 300 is processing
271 / 300 is processing
281 / 300 is processing
291 / 300 is processing


 25%|██▌       | 2/8 [12:07<36:37, 366.28s/it]

...

 75%|███████▌  | 6/8 [36:08<11:55, 357.81s/it]



1 / 300 is processing
11 / 300 is processing
21 / 300 is processing
31 / 300 is processing
41 / 300 is processing
51 / 300 is processing
61 / 300 is processing
71 / 300 is processing
81 / 300 is processing
91 / 300 is processing
101 / 300 is processing
111 / 300 is processing
121 / 300 is processing
131 / 300 is processing
141 / 300 is processing
151 / 300 is processing
161 / 300 is processing
171 / 300 is processing
181 / 300 is processing
191 / 300 is processing
201 / 300 is processing
211 / 300 is processing
221 / 300 is processing
231 / 300 is processing
241 / 300 is processing
251 / 300 is processing
261 / 300 is processing
271 / 300 is processing
281 / 300 is processing
291 / 300 is processing


 88%|████████▊ | 7/8 [41:57<05:54, 354.87s/it]



1 / 300 is processing
11 / 300 is processing
21 / 300 is processing
31 / 300 is processing
41 / 300 is processing
51 / 300 is processing
61 / 300 is processing
71 / 300 is processing
81 / 300 is processing
91 / 300 is processing
101 / 300 is processing
111 / 300 is processing
121 / 300 is processing
131 / 300 is processing
141 / 300 is processing
151 / 300 is processing
161 / 300 is processing
171 / 300 is processing
181 / 300 is processing
191 / 300 is processing
201 / 300 is processing
211 / 300 is processing
221 / 300 is processing
231 / 300 is processing
241 / 300 is processing
251 / 300 is processing
261 / 300 is processing
271 / 300 is processing
281 / 300 is processing
291 / 300 is processing


100%|██████████| 8/8 [47:50<00:00, 358.84s/it]


In [44]:
exp.results

,avg_waiting_time,avg_service_time,avg_total_time,avg_num_in_q,elev1_utilization,elev2_utilization,elev3_utilization,elev4_utilization,elev5_utilization,elev6_utilization,avg_elev_utilization,policy,seed
0,9.866582,74.925815,84.792397,1.186815,0.180892,0.334662,0.341666,0.152658,0.248768,0.176319,0.239161,0,668
0,9.242497,74.726626,83.969123,1.189626,0.260899,0.292256,0.070960,0.244690,0.113638,0.341526,0.220662,0,6482
0,10.954823,81.969217,92.924040,1.253719,0.341798,0.394594,0.339265,0.298842,0.220099,0.164962,0.293260,0,8649
0,11.987649,75.648883,87.636531,1.414412,0.149766,0.214657,0.169522,0.189608,0.300100,0.265805,0.214910,0,8745
0,10.049736,76.070523,86.120259,1.339965,0.218375,0.190478,0.126860,0.185636,0.185540,0.211930,0.186470,0,8733
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10.474398,75.200628,85.675026,1.398969,0.448867,0.273315,0.259661,0.524370,0.088135,0.182529,0.296146,7,3005
0,10.108005,74.246246,84.354251,1.041541,0.260810,0.355822,0.211427,0.211875,0.195941,0.227097,0.243829,7,7926
0,8.880028,69.995305,78.875333,1.225564,0.347239,0.328903,0.293741,0.382601,0.098979,0.204668,0.276022,7,8762
0,8.434804,65.860735,74.295539,1.266589,0.201386,0.340809,0.345529,0.247674,0.146642,0.320733,0.267129,7,5124


# simulation result save

In [45]:
exp.results.to_csv('/content/drive/MyDrive/2023-2 시뮬레이션/results/simulation_result_gen_input.csv');

In [46]:
exp.results.describe()

,avg_waiting_time,avg_service_time,avg_total_time,avg_num_in_q,elev1_utilization,elev2_utilization,elev3_utilization,elev4_utilization,elev5_utilization,elev6_utilization,avg_elev_utilization,policy,seed
count,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000
mean,10.036599,73.389201,83.425800,1.280414,0.279828,0.267457,0.291289,0.267797,0.286862,0.268309,0.276924,3.500000,4933.626667
std,1.153856,6.225548,6.681972,0.153612,0.100431,0.076896,0.149471,0.093844,0.174355,0.102856,0.059555,2.291765,2933.703704
min,6.635309,57.598237,66.689890,0.801526,0.057307,0.046980,0.000000,0.037421,0.000000,0.038044,0.138194,0.000000,4.000000
25%,9.233096,69.039245,78.618440,1.180552,0.214548,0.214234,0.186174,0.200845,0.154461,0.196716,0.234076,1.750000,2360.000000
50%,9.960085,73.066943,83.043406,1.272212,0.264556,0.264922,0.262020,0.258766,0.257810,0.256541,0.276051,3.500000,4942.000000
75%,10.723052,77.498655,87.763715,1.380293,0.323799,0.314572,0.361599,0.322343,0.390839,0.323477,0.319362,5.250000,7740.250000
max,14.778000,94.555838,106.563289,1.899812,0.833230,0.600883,0.961263,0.686362,0.971203,0.762815,0.493632,7.000000,9887.000000
